# Customer Segmentation of Lisbon Hotels Dataset

### Importing libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

%matplotlib inline

In [19]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct, count, when
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

### Initializing spark session & importing dataset

**Task** : Define a schema with proper types as infer schema is not having proper datatypes

In [4]:
spark = SparkSession.builder.appName("My_app").getOrCreate()
sp_df = spark.read.csv('./HotelCustomersDataset.csv', header=True, inferSchema=True)

sp_df.printSchema()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/19 15:01:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- ID: integer (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- DaysSinceCreation: integer (nullable = true)
 |-- NameHash: string (nullable = true)
 |-- DocIDHash: string (nullable = true)
 |-- AverageLeadTime: integer (nullable = true)
 |-- LodgingRevenue: double (nullable = true)
 |-- OtherRevenue: double (nullable = true)
 |-- BookingsCanceled: integer (nullable = true)
 |-- BookingsNoShowed: integer (nullable = true)
 |-- BookingsCheckedIn: integer (nullable = true)
 |-- PersonsNights: integer (nullable = true)
 |-- RoomNights: integer (nullable = true)
 |-- DaysSinceLastStay: integer (nullable = true)
 |-- DaysSinceFirstStay: integer (nullable = true)
 |-- DistributionChannel: string (nullable = true)
 |-- MarketSegment: string (nullable = true)
 |-- SRHighFloor: integer (nullable = true)
 |-- SRLowFloor: integer (nullable = true)
 |-- SRAccessibleRoom: integer (nullable = true)
 |-- SRMediumFloor: integer (nullable = tru

24/04/19 15:02:00 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [5]:
sp_df.show(10)

24/04/19 15:08:44 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+-----------+----+-----------------+--------------------+--------------------+---------------+--------------+------------+----------------+----------------+-----------------+-------------+----------+-----------------+------------------+--------------------+--------------------+-----------+----------+----------------+-------------+---------+--------+------+-------------+---------+--------------+------------------+--------------------+-----------+
| ID|Nationality| Age|DaysSinceCreation|            NameHash|           DocIDHash|AverageLeadTime|LodgingRevenue|OtherRevenue|BookingsCanceled|BookingsNoShowed|BookingsCheckedIn|PersonsNights|RoomNights|DaysSinceLastStay|DaysSinceFirstStay| DistributionChannel|       MarketSegment|SRHighFloor|SRLowFloor|SRAccessibleRoom|SRMediumFloor|SRBathtub|SRShower|SRCrib|SRKingSizeBed|SRTwinBed|SRNearElevator|SRAwayFromElevator|SRNoAlcoholInMiniBar|SRQuietRoom|
+---+-----------+----+-----------------+--------------------+--------------------+----------

## Exploratory Data Analysis

### Check for missing values

In [20]:
missing_counts = sp_df.select([count(when(col(c).isNull(), c)).alias(c) for c in sp_df.columns])
missing_counts.show()

+---+-----------+---+-----------------+--------+---------+---------------+--------------+------------+----------------+----------------+-----------------+-------------+----------+-----------------+------------------+-------------------+-------------+-----------+----------+----------------+-------------+---------+--------+------+-------------+---------+--------------+------------------+--------------------+-----------+
| ID|Nationality|Age|DaysSinceCreation|NameHash|DocIDHash|AverageLeadTime|LodgingRevenue|OtherRevenue|BookingsCanceled|BookingsNoShowed|BookingsCheckedIn|PersonsNights|RoomNights|DaysSinceLastStay|DaysSinceFirstStay|DistributionChannel|MarketSegment|SRHighFloor|SRLowFloor|SRAccessibleRoom|SRMediumFloor|SRBathtub|SRShower|SRCrib|SRKingSizeBed|SRTwinBed|SRNearElevator|SRAwayFromElevator|SRNoAlcoholInMiniBar|SRQuietRoom|
+---+-----------+---+-----------------+--------+---------+---------------+--------------+------------+----------------+----------------+-----------------+--

### Describe statistics of the dataframe

In [6]:
sp_df.describe().show()

+-------+------------------+-----------+------------------+------------------+--------------------+--------------------+-----------------+-----------------+------------------+--------------------+--------------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+-------------------+
|summary|                ID|Nationality|               Age| DaysSinceCreation|            NameHash|           DocIDHash|  AverageLeadTime|   LodgingRevenue|      OtherRevenue|    BookingsCanceled|    BookingsNoShowed| BookingsCheckedIn|    PersonsNights|        RoomNights|DaysSinceLastStay|DaysSinceFirstStay| DistributionChannel|       MarketSegment|        SRHighFloor|      

### Get no. of unique values in each column

In [13]:
unique_counts = sp_df.agg(*(countDistinct(col(c)).alias(c) for c in sp_df.columns))
unique_counts.show()

+-----+-----------+---+-----------------+--------+---------+---------------+--------------+------------+----------------+----------------+-----------------+-------------+----------+-----------------+------------------+-------------------+-------------+-----------+----------+----------------+-------------+---------+--------+------+-------------+---------+--------------+------------------+--------------------+-----------+
|   ID|Nationality|Age|DaysSinceCreation|NameHash|DocIDHash|AverageLeadTime|LodgingRevenue|OtherRevenue|BookingsCanceled|BookingsNoShowed|BookingsCheckedIn|PersonsNights|RoomNights|DaysSinceLastStay|DaysSinceFirstStay|DistributionChannel|MarketSegment|SRHighFloor|SRLowFloor|SRAccessibleRoom|SRMediumFloor|SRBathtub|SRShower|SRCrib|SRKingSizeBed|SRTwinBed|SRNearElevator|SRAwayFromElevator|SRNoAlcoholInMiniBar|SRQuietRoom|
+-----+-----------+---+-----------------+--------+---------+---------------+--------------+------------+----------------+----------------+--------------

### Data Distribution

Make two lists with numerical & categorical columns

In [ ]:
# numerical_columns = [.....]
# categorical_columns = [.....]

Visualize the distribution of numerical variables such as Age, AverageLeadTime, LodgingRevenue, and OtherRevenue using histograms or kernel density plots.

Explore the distribution of categorical variables like DistributionChannel, and MarketSegment using bar plots to understand the frequency of different categories.

In [ ]:
# Plot 1

In [ ]:
# Plot 2

Investigate the distribution of booking-related variables (BookingsCanceled, BookingsNoShowed, BookingsCheckedIn) to understand how often customers cancel bookings or fail to show up.

In [ ]:
# Plot - 3

In [ ]:
# Plot - 4

Visualize the distribution of PersonsNights and RoomNights to understand the typical length of stay and occupancy patterns

In [ ]:
# Plot - 5

In [ ]:
# Plot - 6

Explore the distribution of variables across different market segments and distribution channels to understand how customer preferences vary.

### Distribution of Nationalities (plotly heatmap in world map)

Which country has the most bookings?

Outlier Detection:  
Identify and investigate outliers in numerical variables using box plots or scatter plots. Consider whether these outliers are valid data points or errors that need to be addressed.  
Check for unusual patterns in categorical variables that may indicate data quality issues or anomalies.

### Encoding & Correlation plot

 Before calling oneHotEncoder It's important to reduce the dimensionality of the dataset, and Nationality column has about 187 different unique values, we can drop the countries which occured less than 30 times in the data, and just consider these cases as unregular.

In [ ]:
# remove countries that occur less than 30 times

# cols to be encoded -> ['Nationality', 'MarketSegment', 'DistributionChannel']

## Data Preprocessing

### Get count of null values in each column

### Remove Unimportant Columns

In [ ]:
# Remove Columns -> ['DocIDHash', 'NameHash', 'ID']

## Segmentation using MLlib

Ref: https://spark.apache.org/docs/latest/ml-clustering.html

### KMeans Algorithm

### Gaussian Mixture Model